In [1]:
%config IPCompleter.greedy=True

In [44]:
# TLE
from typing import List

class Solution:
    def findNumOfValidWords(self, words: List[str], puzzles: List[str]) -> List[int]:
        # convert list of string into list of dict (to improve performance, use bool[26] instead of dict())
        def array2dicts(str_arr: List[str]) -> List[List[bool]]:
            L = len(str_arr)
            dicts = [[False] * 26 for _ in range(L)]
            for i, w in enumerate(str_arr):
                for ch in w:
                    dicts[i][ord(ch) - ord('a')] = True
            return dicts
        wdicts = array2dicts(words)
        pdicts = array2dicts(puzzles)
        pfirsts = map(lambda p: ord(p[0])-ord('a'), puzzles)
        res = []
        for pdict, pfirst in zip(pdicts, pfirsts):
            sm = 0
            for wdict in wdicts:
                # condition 1: if puzzle's first letter is in word
                a = wdict[pfirst]
                # condition 2: if word's every letter is in puzzle
                # `not w` means this letter doesn't exist in word
                # `or (w and p)` means this letter must exist in both word and puzzle
                b = all((not w) or (w and p) for p, w in zip(pdict, wdict))
                sm += (a and b)
            res.append(sm)
        return res

In [58]:
# https://leetcode.com/problems/number-of-valid-words-for-each-puzzle/discuss/371864/Python-Solution
# Runtime: 828 ms, faster than 73.58%
from typing import List
from collections import Counter
import itertools

class Solution:
    def findNumOfValidWords(self, words: List[str], puzzles: List[str]) -> List[int]:
        # Part I
        # 1. for each word, calculate the set of word's letters
        # 2. count the number of each unique set, save to a counter, e.g. 'a', 'aa' -> {frozenset('a'): 2}
        # Step 2 dedupes the words, and greatly reduces the input number
        # T: O(W), S: O(W)
        count = Counter(frozenset(w) for w in words)
        # Part II
        # 1. calculate all subsets of puzzle[1:]
        # 2. for each subset of puzzle[1:], check occurrence of frozenset(puzzle[0] + subset) in count above
        # 3. add occurrence to final sum
        # T: O(2^6 * P), S: O(1)
        # 6 = len(puzzle) - first_letter = 7 - 1
        # 2^n is for all combinations
        ret = []
        for p in puzzles:
            cnt = 0
            for i in range(len(p)): # index starting from 0 because we need to consider the case to concat p[0]
                for tpl in itertools.combinations(p[1:], i):  # i is expected length (from 0 to L-1)
                    cnt += count[frozenset((p[0],) + tpl)]  # (p[0],) equals to tuple(p[0])
            ret += cnt,  # `ret += cnt,` equals to `ret.append(cnt)`
        return ret

In [70]:
# Same from https://leetcode.com/problems/number-of-valid-words-for-each-puzzle/discuss/371864/Python-Solution
# BFS + Bitmask solution
# Runtime: 772 ms, faster than 77.36%
from typing import List
from collections import Counter

class Solution:
    def findNumOfValidWords(self, words: List[str], puzzles: List[str]) -> List[int]:
        def letter_index_binary(ch: str) -> int:
            return 1 << (ord(ch) - ord('a'))
        # Part I
        cnt = Counter()
        for w in words:
            mask = 0
            for ch in w:
                # set existing letter digit as 1. Total mask digit number: 26
                # e.g. 'ac' -> 00....0101
                mask |= letter_index_binary(ch)
            cnt[mask] += 1
        # Part II
        res = []
        for p in puzzles:
            # BFS to traverse all combinations
            bfs = [letter_index_binary(p[0])]
            for ch in p[1:]:
                bfs += [mask | letter_index_binary(ch) for mask in bfs]
            res += sum(cnt[mask] for mask in bfs),
        return res

In [71]:
Solution().findNumOfValidWords(words = ["aaaa","asas","able","ability","actt","actor","access"], puzzles = ["aboveyz","abrodyz","abslute","absoryz","actresz","gaswxyz"])

[1, 1, 3, 2, 4, 0]